In [1]:
import pandas as pd
df = pd.read_csv("India_Must_See_Places.csv")
df.head()

,Unnamed: 0,Zone,State,City,Name,Type,Establishment Year,time needed to visit in hrs,Google review rating,Entrance Fee in INR,Airport with 50km Radius,Weekly Off,Significance,DSLR Allowed,Number of google review in lakhs,Best Time to visit
0,0,Northern,Delhi,Delhi,India Gate,War Memorial,1921,0.5,4.6,0,Yes,NaN,Historical,Yes,2.60,Evening
1,1,Northern,Delhi,Delhi,Humayun's Tomb,Tomb,1572,2.0,4.5,30,Yes,NaN,Historical,Yes,0.40,Afternoon
2,2,Northern,Delhi,Delhi,Akshardham Temple,Temple,2005,5.0,4.6,60,Yes,NaN,Religious,No,0.40,Afternoon
3,3,Northern,Delhi,Delhi,Waste to Wonder Park,Theme Park,2019,2.0,4.1,50,Yes,Monday,Environmental,Yes,0.27,Evening
4,4,Northern,Delhi,Delhi,Jantar Mantar,Observatory,1724,2.0,4.2,15,Yes,NaN,Scientific,Yes,0.31,Morning


In [7]:
def distance_score(source_city, source_state, source_zone, row):
    if row["City"] == source_city:
        return 1.0
    elif row["State"] == source_state:
        return 0.85
    elif row["Zone"] == source_zone:
        return 0.65
    else:
        return 0.2

In [8]:
def rank_weekend_destinations(source_city):
    source_row = df[df["City"] == source_city].iloc[0]
    source_state = source_row["State"]
    source_zone = source_row["Zone"]

    temp_df = df.copy()

    temp_df["distance_score"] = temp_df.apply(
        lambda row: distance_score(
            source_city,
            source_state,
            source_zone,
            row
        ),
        axis=1
    )

    temp_df["rating_norm"] = temp_df["Google review rating"] / temp_df["Google review rating"].max()
    temp_df["popularity_norm"] = (
        temp_df["Number of google review in lakhs"] /
        temp_df["Number of google review in lakhs"].max()
    )

    temp_df["final_score"] = (
    0.45 * temp_df["distance_score"] +
    0.35 * temp_df["rating_norm"] +
    0.20 * temp_df["popularity_norm"]
    )

    result = (
        temp_df[temp_df["City"] != source_city]
        .sort_values(by="final_score", ascending=False)
        .head(5)
    )

    return result[["Name", "City", "State", "final_score"]]

In [11]:
city = input("Enter source city: ")

recommendations = rank_weekend_destinations(city)

recommendations


Enter source city: Delhi


,Name,City,State,final_score
305,Gurudwara Bangla Sahib,New Delhi,Delhi,0.753736
313,Jama Masjid,New Delhi,Delhi,0.717172
318,Rail Museum,New Delhi,Delhi,0.703272
92,Golden Temple (Harmandir Sahib),Amritsar,Punjab,0.693851
306,Kedarnath,Kedarnath,Uttarakhand,0.689411
